# Convex Hulls

General information

## Table of Contents

1. Introduction
2. Imports
3. Algorithms
    * Naive Hull
    * Graham Scan
    * Gift Wrapping
    * Chan's Hull
4. Conclusion

## 2. Imports

In [ ]:
from geometry import Point, PointRef, Orientation

In [ ]:
from visualisation import Visualisation
vis = Visualisation()
points = [Point(10,10), Point(100,50), Point(250,25), Point(100,100), Point(200,200), Point(300,10), Point(400,10)]
vis.add_points(points)
vis.register_instance("Test instance", points)

## 3. Algorithms

### 3.1 Naive Hull
The first algorithm from the lecture

In [ ]:
def naive_hull(points: list[Point]) -> list[Point]:
    edges: dict[Point, Point] = {}
    for p in points:
        for q in filter(lambda q: q != p, points):
            valid = True
            for r in filter(lambda r: r != p and r != q, points):
                orient = r.orientation(p, q)
                if orient is Orientation.RIGHT or orient is Orientation.BEHIND_SOURCE or orient is Orientation.BEHIND_TARGET:
                    valid = False
                    break
            if valid:
                edges[p] = q
    hull: list[Point] = []
    if edges:
        first_point, next_point = edges.popitem()
        hull.append(first_point)
        while next_point is not None and next_point != first_point:
            hull.append(next_point)
            next_point = edges.pop(next_point, None)
    return hull

In [ ]:
hull = naive_hull(points)
print(hull)

In [ ]:
vis.register_algorithm("Naive hull", naive_hull)

In [ ]:
vis.display()

should handle degeneracies, but it's not robust

### 3.2 Graham Scan
Better algorithm

In [ ]:
def graham_scan(points: list[Point]) -> list[Point]:    
    if len(points) <= 2:
        return points[:]
    sorted_points = sorted(points, key = lambda p: (p.x, p.y))
    upper_hull = graham_half_scan(sorted_points)
    sorted_points.reverse()
    lower_hull = graham_half_scan(sorted_points)
    hull = upper_hull[:-1] + lower_hull[:-1]
    return hull

def graham_half_scan(points: list[Point]) -> list[Point]:
    if len(points) <= 2:
        return points[:]
    hull = points[:2]
    for p in points[2:]:
        hull.append(p)
        while len(hull) > 2 and hull[-2].orientation(hull[-3], hull[-1]) is not Orientation.LEFT:
            del hull[-2]
    return hull

In [ ]:
print(graham_scan([Point(0,0), Point(1,0.5), Point(2.5,0.25), Point(1,1), Point(2,2), Point(3,0), Point(4,0)]))

In [ ]:
vis.register_algorithm("Graham scan", graham_scan)

In [ ]:
vis.display()

should handle degeneracies and be somewhat robust

### 3.3 Gift Wrapping
Also called *Jarvis's March*. Output-sensitive.

In [ ]:
def gift_wrapping(points: list[Point]) -> list[Point]:
    if len(points) <= 2:        # wie umgehen mit Duplikaten ? evtl. set nehmen...
        return points[:]
    first_point = max(points, key = lambda p: (p.x, p.y))
    wrapper = Wrapper(first_point)
    hull = [first_point]
    while True:
        next_point = wrapper.get_next_point(points)
        if next_point == first_point:
            break             # sicherstellen, dass dieser Punkt erreicht wird
        hull.append(next_point)
    return hull


class Wrapper:
    def __init__(self, current_point: Point):
        self.current_point = current_point
    
    def get_next_point(self, points: list[Point]) -> Point:
        points_iter = filter(lambda p: p != self.current_point, points)
        next_point = next(points_iter)
        for p in points_iter:
            if p != next_point and self.is_better(p, next_point):         # wie umgehen mit Duplikaten ?
                next_point = p
        self.current_point = next_point
        return next_point

    def is_better(self, p: Point, q: Point) -> bool:
        orient = p.orientation(self.current_point, q)
        return orient is Orientation.LEFT or orient is Orientation.BEHIND_TARGET

In [ ]:
print(gift_wrapping([Point(0,0), Point(1,0.5), Point(2.5,0.25), Point(1,1), Point(2,2), Point(3,0), Point(4,0)]))

In [ ]:
vis.register_algorithm("Gift wrapping", gift_wrapping)

In [ ]:
vis.display()

### 3.4 Chan's Hull
Best of both worlds

In [ ]:
from typing import Optional
    
    
class ChanWrapper(Wrapper):
    def __init__(self, current_point: PointRef):
        super(ChanWrapper, self).__init__(current_point)
    
    def has_same_angle(self, p: Point, q: Point) -> bool:
        orient = p.orientation(self.current_point, q)
        return orient is Orientation.BETWEEN or orient is Orientation.BEHIND_TARGET
    
    # Handles all the degenerate edge cases. Don't read it, lest it will haunt you in your nightmares.
    def calculate_indexes(self, mini_hull: list[Point], l: int, r: int) -> tuple[int, int, int, int, int]:
        if l != r and self.has_same_angle(mini_hull[l], mini_hull[r]):
            if self.is_better(mini_hull[r], mini_hull[l]):
                l += 1
            else:
                r -= 1
        if r - l < 2:
            return l, 0, 0, 0, r
        m = int(l + (r - l) / 2)
        mb = m-1
        ma = m+1
        if self.has_same_angle(mini_hull[m], mini_hull[mb]):
            if self.is_better(mini_hull[mb], mini_hull[m]):
                m -= 1
            mb -= 1
        elif self.has_same_angle(mini_hull[m], mini_hull[ma]):
            if self.is_better(mini_hull[ma], mini_hull[m]):
                m += 1
            ma += 1
        return l, mb, m, ma, r
    
    # Can return more than one candidate but at most five which is asymptotically constant.
    def get_candidates(self, mini_hull: list[Point]) -> list[PointRef]:
        if self.current_point.is_in_container(mini_hull):
            if len(mini_hull) == 1:
                return []
            next_position = (self.current_point.get_position() + 1) % len(mini_hull)
            return [PointRef(mini_hull, next_position)]
        l, mb, m, ma, r = self.calculate_indexes(mini_hull, 0, len(mini_hull) - 1)
        while l <= mb < m < ma <= r:
            m_better_than_before = self.is_better(mini_hull[m], mini_hull[mb])
            m_better_than_after = self.is_better(mini_hull[m], mini_hull[ma])
            if m_better_than_before and m_better_than_after:
                return [PointRef(mini_hull, m)]
            m_better_than_l = self.is_better(mini_hull[m], mini_hull[l])
            r_better_than_l = self.is_better(mini_hull[r], mini_hull[l])
            if r_better_than_l:
                if m_better_than_l and not m_better_than_before:
                    r = mb
                else:
                    l = m
            else:
                if m_better_than_l and m_better_than_before:
                    l = ma
                else:
                    r = m
            l, mb, m, ma, r = self.calculate_indexes(mini_hull, l, r)
        return [PointRef(mini_hull, pos) for pos in range(l, r+1)]

    
def chans_hull_m(points: list[Point], m: int) -> Optional[list[Point]]:
    mini_hulls = []
    mini_hull_points = []
    for i in range(0, len(points), m):
        j = min(len(points), i+m)
        mini_hull = graham_scan(points[i:j])
        mini_hulls.append(mini_hull)
        for position in range(0, len(mini_hull)):
            mini_hull_points.append(PointRef(mini_hull, position))
    first_point = max(mini_hull_points, key = lambda p: (p.x, p.y))
    wrapper = ChanWrapper(first_point)
    hull = [first_point]
    for _ in range(m):
        candidates = []
        for mini_hull in mini_hulls:
            candidates.extend(wrapper.get_candidates(mini_hull))
        next_point = wrapper.get_next_point(candidates)
        if next_point == first_point:
            return hull
        hull.append(next_point)
    return None

def chans_hull(points: list[Point]) -> list[Point]:
    if len(points) <= 2:
        return points[:]
    hull = None
    t = 0
    while hull is None:
        m = min(len(points), 2**(2**t))
        hull = chans_hull_m(points, m)
        t += 1
    return hull

In [ ]:
print(chans_hull([Point(0,0), Point(1,0.5), Point(2.5,0.25), Point(1,1), Point(2,2), Point(3,0), Point(4,0), Point(1,2), Point(3,1.5)]))

In [ ]:
print(chans_hull([Point(0,3), Point(10,10), Point(20,5)]))

In [ ]:
vis.register_algorithm("Chan's hull", chans_hull)

In [ ]:
vis.display()